## [Sundail (LLM) versus Temporal Convolutional Networks (TCN) for Time Series Forecasting of Crude Oil Pricing](https://medium.com/@kylejones_47003/sundail-llm-versus-temporal-convolutional-networks-tcn-for-time-series-forecasting-of-crude-oil-a954d5e2f9d2)

Sundial is a causal transformer designed for time series. It treats the sequence like a language modeling task. The TCN is a 1D convolutional model with a single kernel layer, ReLU, and linear output.

Sundial handles uncertainty well and provides a probabilistic forecast. The TCN responds more directly to recent fluctuations and produces a tighter prediction. Both learn meaningful structure from the same inputs. Both capture broad seasonal shape. Their residuals differ in character.

In [1]:
!pip install -q numpy pandas scikit-learn matplotlib
!pip install -q pandas-datareader torch transformers

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas_datareader.data import DataReader
from datetime import datetime
from transformers import AutoModelForCausalLM
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [4]:
def fetch_fred_data(series_id, start='2000-01-01'):
    df = DataReader(series_id, 'fred', start)
    df = df.rename(columns={series_id: 'value'})
    df = df.dropna()
    df = df.reset_index()
    return df

def compute_mape(y_true, y_pred):
    eps = 1e-8
    ape = torch.abs((y_true - y_pred) / (y_true + eps))
    ape = torch.where(torch.isinf(ape), torch.zeros_like(ape), ape)
    return ape.mean().item() * 100

In [5]:
class TCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, kernel_size=3):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, hidden_size, kernel_size, padding=2)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.mean(dim=2)
        return self.fc(x)

In [6]:
# Reload everything and redo forecasting
df = fetch_fred_data('DCOILWTICO')
df = df.set_index('DATE').resample('D').mean().dropna()

raw_series = df['value']
mean = raw_series.mean()
std = raw_series.std()
series = (raw_series - mean) / std
series = series.dropna()

lookback_length = 256
forecast_length = 96
num_samples = 20

train_values = series.values[-(lookback_length + forecast_length + 100):-(forecast_length)]
true_future = raw_series.values[-forecast_length:]
true = torch.tensor(true_future)

print(f"Mean: {mean}")
print(f"Standard Deviation: {std}")

Mean: 63.84029563585172
Standard Deviation: 25.230146245915975


In [7]:
# Sundial
input_tensor = torch.tensor(train_values[-lookback_length:], dtype=torch.float32).unsqueeze(0)
sundial = AutoModelForCausalLM.from_pretrained('thuml/sundial-base-128m', trust_remote_code=True)
sundial_samples = [sundial.generate(input_tensor, max_new_tokens=forecast_length) for _ in range(num_samples)]
sundial_output = torch.stack(sundial_samples)

sundial_pred_norm = sundial_output.mean(dim=0).squeeze()[:forecast_length]
sundial_pred = sundial_pred_norm * std + mean
lower = sundial_output.quantile(0.10, dim=0).squeeze()[:forecast_length] * std + mean
upper = sundial_output.quantile(0.90, dim=0).squeeze()[:forecast_length] * std + mean

print(f"Sundail Predictions: {sundial_pred}")
print(f"Lower Quantile: {lower}")
print(f"Upper Quantile: {upper}")

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


AttributeError: 'DynamicCache' object has no attribute 'get_max_length'

In [ ]:
# TCN
X_seq, y_seq = [], []
for i in range(len(train_values) - lookback_length - forecast_length):
    X_seq.append(train_values[i:i+lookback_length])
    y_seq.append(train_values[i+lookback_length:i+lookback_length+forecast_length])

X_seq = torch.tensor(X_seq, dtype=torch.float32).unsqueeze(1)
y_seq = torch.tensor(y_seq, dtype=torch.float32)
loader = DataLoader(TensorDataset(X_seq, y_seq), batch_size=16, shuffle=True)

tcn_model = TCN(input_size=1, hidden_size=32, output_size=forecast_length)
optimizer = torch.optim.Adam(tcn_model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(200):
    tcn_model.train()
    for xb, yb in loader:
        optimizer.zero_grad()
        out = tcn_model(xb)
        loss = loss_fn(out, yb)
        loss.backward()
        optimizer.step()

tcn_model.eval()
last_input = torch.tensor(train_values[-lookback_length:], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
tcn_pred_norm = tcn_model(last_input).detach().squeeze()
tcn_pred = tcn_pred_norm * std + mean

In [8]:
# Plot
def plot_full_forecast(df, true, sundial_pred, tcn_pred, lower, upper, forecast_length):
    past_2y = df[-(730 + forecast_length):-forecast_length]
    future_dates = df.index[-forecast_length:]

    plt.figure(figsize=(14, 5))
    plt.plot(past_2y.index, past_2y.values, label="Historical (2y)", linewidth=1.8)
    plt.plot(future_dates, true.numpy(), label="True", linewidth=2)
    plt.plot(future_dates, sundial_pred.detach().numpy(), label="Sundial", linestyle="--")
    plt.plot(future_dates, tcn_pred.detach().numpy(), label="TCN", linestyle=":")
    plt.fill_between(future_dates,
                     lower.detach().numpy(),
                     upper.detach().numpy(),
                     color='blue', alpha=0.2, label="Sundial 80% CI")
    plt.title("Two-Year Historical Data with Forecast Comparison")
    plt.legend()
    plt.tight_layout()
    ## plt.savefig("forecast_2yr_plus_predictions.png")
    plt.show()

In [9]:
plot_full_forecast(df['value'], true, sundial_pred, tcn_pred, lower, upper, forecast_length)

NameError: name 'sundial_pred' is not defined